<img src="swung_logo_vector.png" alt="swung" style="width: 40%;"/>

# T21 Segysak Tutorial - Tony Hallam, April 2021



# Introduction

## What is `segysak`?

## Tour

## `segysak` vs X

## SEG-Y Files

## `segysak`

Make SEG-Y data easily accessible and creatable from Python

Simply put, `segysak` has grown as a set of tools to make SEG-Y data easily accessible and createable from Python.
If leverages a number of existing libraries but brings them together to try and improve the user experience, and
to remove as much boiler plate code as possible when dealing with SEG-Y.

The project started about a year ago at Transform 2020. Most of the work was done during that hackathon but it has
continued to develop since then with gradual improvements, bug-fixes and user support.

Although I'm the project owner and one of the primary users of `segysak` (I use it it in a lot of my PhD projects).
It is open for the subsurface community to not only utilise, but to contribute to grow to meet peoples needs.

I'd strongly encourage anyone with ideas and/or enthusiasm for changes or additions to get in touch so we can improve `segysak` for everyone.

## Tour

 - Github (source code, issues, contributions) - https://github.com/trhallam/segysak
 - Documentation (help, examples, API) - https://segysak.readthedocs.io/en/latest/
 - Slack (help, discussion, ideas, contributions) - https://swung.slack.com/messages/segysak/

Everything you need to know about `segysak` is available online. There is the Github repository where we manage the source code for the library and distribute the packages for installation via pip. 

There is also an issue tracker where you can raise bugs/problems or submit ideas or suggestions. It's also a good place to look for things that need doing if you want to help out.

We then have the documentation on readthedocs. Here you will find more detailed help, examples (which are avaialble as Notebooks) and the API (of function and member descriptions). This is a really useful place to come if you are stuck, or
need more detail because we cover a lot of the basics in the documentation. Indeed this workshop is heavily influenced
by the first few example notebooks you can find here.

Finally, we have the Slack forum hosted on swung.slack. This space is always open for people to ask questions or get help, even drop by just for a bit of discussion.

## `segysak` versus X

A lot of the time I get asked about segysak versus X in the Python world, where does it fit in?
The reality is, segysak doesn't so much compete with any part of the scientific stack but tries to form bridges over
the common space we often have to traverse. For example.

### `segyio`

 - `segysak` relies on `segyio` but abstracts a lot of the low level detail

segysak couldn't exist without segyio - segyio does all the hard work of interacting with the actual SEG-Y and segysak tries to make segyio a bit more accessible by providing a direct link between it and easy to use libraries like xarray.

### `xarray`

 - `segysak` extends `xarray` to make it easier to deal with SEG-Y files

Things like loading and writing of files are more automated. Trys to take care of tracking things like headers, and attributes for you.

Also includes extensions for common seismic related tasks.

## SEG-Y Files

File format defined by the SEG Organisation for storing seismic trace data.

Heavily geared toward limited size magnetic reel tapes.

**Basic Format (SEGY-Rev2):**

<img src="segy_layout.png" alt="swung" style="width: 100%;"/>


# Installation

`pip install segysak`

Demo Data

Opening the Tutorial Notebook

In [ ]:
from segysak.segy import segy_loader


## Basic Imports

In [ ]:
import pathlib
from IPython.display import display
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Basic Usage

### Inspecting SEG-Y files

### Loading SEG-Y files

### `xarray.Dataset` basics

### NetCDF Files

### Editing and Saving

In [ ]:
# specify the example file and check we have the example data

segy_file = pathlib.Path("data/volve10r12-full-twt-sub3d.sgy")
print("SEG-Y exists:", segy_file.exists())
 

## Inspecting SEG-Y files


In [ ]:
from segysak.segy import segy_header_scan, segy_header_scrape, get_segy_texthead

In [ ]:
# examine the text header
get_segy_texthead(segy_file)

## Inspecting SEG-Y files - trace header scan


In [ ]:
# scan the headers to check
scan = segy_header_scan(segy_file, max_traces_scan=2000)
with pd.option_context("display.max_rows", 100):
    display(scan)

Use the context manager

with pd.option_context("display.max_rows", 100):
    display(scan)
    
index names are the same as what `segyio` uses

point out byte locations - segysak preference

## Inspecting SEG-Y files - trace header scrape


In [ ]:
trace_headers = segy_header_scrape(segy_file)
trace_headers

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(20, 10), sharex=True)

for ax, prop in zip(axs.ravel(), ["CDP_X", "CDP_Y", "INLINE_3D", "CROSSLINE_3D"]):
    ax.plot(trace_headers[prop])
    ax.set_title(prop, fontdict={"fontsize":18})

for ax in axs[1, :]:
    ax.set_xlabel("Trace", fontdict={"fontsize":18})

Ranges look reasonable.

Sequencing and patterns look good.

Probably have right byte locations.

### Loading SEG-Y files - complete



In [ ]:
from segysak.segy import segy_loader
help(segy_loader)

sdfasdf

In [ ]:
seisnc_vol = segy_loader(segy_file)

In [ ]:
print(seisnc_vol)

In [ ]:
# lets checkout our data
_ = seisnc_vol.sel(iline=10100).data.T.plot(yincrease=False, figsize=(20, 10), vmax=10)

In [ ]:
# specifying byte locations for key cube geometry
seisnc_vol = segy_loader(segy_file, iline=189, xline=193, cdpx=181, cdpy=185)

In [ ]:
print(seisnc_vol)

In [ ]:
# extra_byte_fields (list/mapping)
seisnc_vol = segy_loader(segy_file, iline=189, xline=193, cdpx=181, cdpy=185, extra_byte_fields=[73, 77], )# {"source":77})

In [ ]:
print(seisnc_vol)

### Loading SEG-Y files - with filtering



In [ ]:
seisnc_vol_iline_10100 = segy_loader(segy_file, ix_crop=(10100, 10100, 2000, 3000))

In [ ]:
print(seisnc_vol_iline_10100)

Sometimes SEGY are really big and we may not want to load the whole thing.
Filtering can help us get around this.

In [ ]:
seisnc_vol_iline_block = segy_loader(segy_file, head_df=trace_headers[trace_headers["INLINE_3D"] <= 10100].copy())

In [ ]:
print(seisnc_vol_iline_block)

Filtering of the `trace_headers` can be done in any normal way using `pandas`. 

Don't change the columns or the index.

Point out sister function `segy_converter`

## `xarray.Dataset` basics

Based upon the NetCDF file format for multi-variable, n-dimensional data.

<center> <img src="seisnc-diagram.png" alt='seisnc' width="50%" /> </center>

In [ ]:
# dataset anatomy - dimensions, coordinates, variables, attributes - DataArray vs Dataset
seisnc_vol

In [ ]:
# data selection - sel, isel

In [ ]:
# data as numpy array
seisnc_vol.iline.values

In [ ]:
# xarray methods (plot, interp, mean, min, max, transpose, broadcast_like, etc...)
# seisnc_vol.cdp_x.transpose("xline", "iline")

In [ ]:
# xarray variable assignment

## `xarray` FAQ

 - Why don't we make the global coordinates the dimensions?
 - How do I save/persist my changes.

Notes: 
Global coordinates are not orthogonal because the seismic grid rarely lines up with Grid North.
Persisting changes either means saving back to SEG-Y or using the NetCDF File Format. 

# NetCDF File Format

Common in climate science, binary, fast and lazy loading

(basically `xarray.Datataset` on disk)


## NetCDF

Why use “another” file format for seismic?

 - Faster than SEG-Y for most use cases.

 - Widely supported within the Python scientific stack (xarray, dask).

 - Commonly supported in other languages.

Generally it just makes working with seismic in Python easier. It will save you time if you are reading volumes repeatedly or can't store everything you need in memory.

NetCDF was the logical choice because it is at the core of `xarray` but `xarray` supports other data models such as zarr which are investigating.
There is also beta support within `segysak` for the OpenZGY format with instructions on Github about how to set that up.

In [ ]:
# output the data to netcdf
seisnc_vol.seisio.to_netcdf("data/test.seisnc")

In [ ]:
## linux
# !ls data/.

## windows
!dir data\.

Notes:

This creates a seisnc file which is slightly smaller than the original segy. 

In [ ]:
from segysak import open_seisnc
open_seisnc("data/test.seisnc")

Notes:

To open a seisnc file we just use the `open_seisnc` method from segysak.
`open_seisnc` is a thin wrapper around the `xarray.open_dataset` method that includes
some special handling for segysak attributes and ensures that the dataset is opened
with the `.seis` extension for xarray which we will talk about soon.

# Editing Data and Saving to SEG-Y

 - Apply a function to headers (adjust values as DF).
 - Apply a function to data (mask/filter).
 - Return to segy highlighting requirements of segysak.


Saving the data to netcdf requires the use of the seisio accessor due to limitations on the types of attributes that can be
saved using the xarray method.



In [ ]:
from segysak.segy import segy_writer
help(segy_writer)

In [ ]:
# export in memory dataset to segy
segy_writer(seisnc_vol, "data/test.segy")

In [ ]:
## linux
# !ls data/.

## windows
!dir data\.

# 10 Minute Break

In [ ]:
import time
from tqdm.auto import tqdm

with tqdm(desc="Break Timer", total=10*60, bar_format="{l_bar}{bar} {elapsed_s:.0f}/{total} seconds") as pbar:
    start = time.time()
    now = time.time()
    prev_now = now
    while (now - start) < 10*60:
        pbar.update(now - prev_now)
        time.sleep(1)
        prev_now = now
        now = time.time()
    pbar.update(time.time() - prev_now)

# Horizon extraction

 - Load a horizon and add it to a cube
 - Plotting maps
 - Plotting horizons on vertical slices
 - Masking and sampling around a horizon


## Load some seismic horizon data

In [ ]:
top_hugin_path = pathlib.Path("data/hor_twt_hugin_fm_top.dat")
print("File", top_hugin_path, "exists?", top_hugin_path.exists())

In [ ]:
# check the file layout
with open(top_hugin_path) as f:
    lines = [next(f) for i in range(5)]
print(*lines)

In [ ]:
# is a csv file
top_hugin_df = pd.read_csv(top_hugin_path, names=["cdp_x","cdp_y","twt"], sep=' ')
top_hugin_df.head()

In [ ]:
top_hugin_ds = seisnc_vol.seis.surface_from_points(top_hugin_df, 'twt', right=('cdp_x', 'cdp_y'))
top_hugin_ds

In [ ]:
top_hugin_ds.twt.plot(cmap='hsv')

In [ ]:
tform = seisnc_vol.seis.get_affine_transform()

In [ ]:
axs = plt.subplot()
mesh = axs.pcolormesh(
    top_hugin_ds.cdp_x.values,
    top_hugin_ds.cdp_y.values,
    top_hugin_ds.twt.values,
    shading="auto"
)
axs.set_aspect(1)
axs.plot([10100, 10100], [2200, 2300], transform=tform + axs.transData, color="w")

In [ ]:
axs = plt.subplot()
mesh = axs.pcolormesh(
    top_hugin_ds.iline.values,
    top_hugin_ds.xline.values,
    top_hugin_ds.twt.T.values,
    shading="auto",
    transform=tform + axs.transData
)
axs.set_aspect(1)
axs.plot([10100, 10100], [2200, 2300], transform=tform + axs.transData, color="w")

In [ ]:
# assign horizon back to seismic
seisnc_vol["hugin"] = top_hugin_ds.twt
seisnc_vol

In [ ]:
# plotting
fig, axs = plt.subplots(figsize=(20, 5))
seisnc_vol.sel(iline=10100, twt=range(2402, 2900, 4), method='nearest').data.T.plot(ax=axs, yincrease=False)
axs.plot(seisnc_vol.sel(iline=10100).xline, seisnc_vol.sel(iline=10100).hugin, 'k')

# Mapping functions over blocks

 - Learn how to use Xarray to map functions on blocks of data, such as trace maths.

Horizon Flattening?

In [ ]:
seisnc_vol["trace"] = (("iline", "xline"), np.arange(61*202, dtype=int).reshape(61, 202))

In [ ]:
def hflat(ds, hor_var, twt_out):
    trace_out = ds.copy()
    trace_out["twt"] = ds.twt - np.squeeze(ds[hor_var].values)
    return trace_out.data.interp(twt=twt_out)

In [ ]:
flat_twt = np.arange(-seisnc_vol.hugin.max(), seisnc_vol.twt.max()-seisnc_vol.hugin.min(), 1, dtype=int)

In [ ]:
tg_gby = seisnc_vol.sel(iline=10100).groupby("trace").map(hflat, args=("hugin", flat_twt))

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(20, 5))
seisnc_vol \
    .sel(iline=10100, twt=range(2002, 2900, 4), method='nearest') \
    .data.T.plot(ax=axs[0], yincrease=False)
axs[0].plot(seisnc_vol.sel(iline=10100).xline, seisnc_vol.sel(iline=10100).hugin, 'k')
tg_gby \
    .sel(twt=range(-300, 300, 4), method='nearest') \
    .T.plot(ax=axs[1], yincrease=False)
axs[1].hlines(0, 0, 10000, "k")

In [ ]:
seisnc_vol_chkd = seisnc_vol.chunk({"iline":1, "xline":1})
template = seisnc_vol_chkd.interp(twt=flat_twt)
tg_mb = seisnc_vol_chkd.map_blocks(hflat, args=("hugin", flat_twt), template=template.data)

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(20, 5))
seisnc_vol.sel(iline=10100, twt=range(2002, 2900, 4), method='nearest').data.T.plot(ax=axs[0], yincrease=False)
tg_mb.sel(iline=10100, twt=range(-300, 300, 4), method='nearest').T.plot(ax=axs[1], yincrease=False)

In [ ]:
%time tg_mb = tg_mb.compute()

# Vectorization of Seismic

 - I want to do machine learning and I need to tabularize my seismic and headers.
 - Now I need to send my results back to SEG-Y.


In [ ]:
seisnc_vol_df = seisnc_vol.to_dataframe()

In [ ]:
print(seisnc_vol_df)

In [ ]:
seisnc_reindex = seisnc_vol_df.reset_index()
print(seisnc_reindex)

In [ ]:
seisnc_df_multi = seisnc_reindex.set_index(["iline", "xline", "twt"])
print(seisnc_df_multi)

In [ ]:
seisnc_xr = seisnc_df_multi.to_xarray()
print(seisnc_xr)

'cdp_x' and 'cdp_y' have come back as 3d cubes, need to reset that
all the seisnc attributes are missing

In [ ]:
seisnc_xr.attrs = seisnc_vol.attrs
display(seisnc_xr.attrs)

In [ ]:
seisnc_xr["cdp_x"] = seisnc_xr["cdp_x"].mean(dim=["twt"])
seisnc_xr["cdp_y"] = seisnc_xr["cdp_y"].mean(dim=["twt"])
seisnc_xr = seisnc_xr.set_coords(["cdp_x", "cdp_y"])
print(seisnc_xr)

# Questions - Slack time because we will run over.

 - Fall backs to chat about memory management, dask, other file formats such as ZGY and Zarr
 - Demo of CLI for quick looks at headers or EBCIDC
 - Contribution Opportunities / Community led development
